In [1]:
# FP_tree的类定义
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue #节点名字
        self.count = numOccur #节点计数值
        self.nodeLink = None #用于链接相似的元素项
        self.parent = parentNode      #needs to be updated
        self.children = {} #子节点

    def inc(self, numOccur):
        '''
        对count变量增加给定值
        '''
        self.count += numOccur

    def disp(self, ind=1):
        '''
        将树以文本形式展示
        '''
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

# FP_tree构建函数
def createTree(dataSet, minSup=1):
    '''
    创建FP树
    '''
    headerTable = {}
    #第一次扫描数据集
    for trans in dataSet:#计算item出现频数
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    headerTable = {k:v for k,v in headerTable.items() if v >= minSup}
    freqItemSet = set(headerTable.keys())
    #print ('freqItemSet: ',freqItemSet)
    if len(freqItemSet) == 0: return None, None  #如果没有元素项满足要求，则退出
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #初始化headerTable
    #print ('headerTable: ',headerTable)
    #第二次扫描数据集
    retTree = treeNode('Null Set', 1, None) #创建树
    for tranSet, count in dataSet.items():  
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#将排序后的item集合填充的树中
    return retTree, headerTable #返回树型结构和头指针表

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#检查第一个元素项是否作为子节点存在
        inTree.children[items[0]].inc(count) #存在，更新计数
    else:   #不存在，创建一个新的treeNode,将其作为一个新的子节点加入其中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #更新头指针表
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#不断迭代调用自身，每次调用都会删掉列表中的第一个元素
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

def updateHeader(nodeToTest, targetNode):
    '''
    this version does not use recursion
    Do not use recursion to traverse a linked list!
    更新头指针表，确保节点链接指向树中该元素项的每一个实例
    '''
    while (nodeToTest.nodeLink != None):    
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

# 抽取条件模式基
def ascendTree(leafNode, prefixPath): #迭代上溯整棵树
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

# 递归查找频繁项集
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]# 1.排序头指针表
    for basePat in bigL:  #从头指针表的底端开始
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        print ('finalFrequent Item: ',newFreqSet)    #添加的频繁项列表
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        print ('condPattBases :',basePat, condPattBases)
        # 2.从条件模式基创建条件FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
#         print ('head from conditional tree: ', myHead)
        if myHead != None: # 3.挖掘条件FP树
            print ('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [2]:
# 测试2
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
            #    ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):  
    retDict = {}  
    for trans in dataSet:  
        retDict[frozenset(trans)] = retDict.get(frozenset(trans), 0) + 1 #若没有相同事项，则为1；若有相同事项，则加1  
    return retDict

minSup = 2
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
myFPtree, myHeaderTab = createTree(initSet, minSup)
myFPtree.disp()
myFreqList = []
mineTree(myFPtree, myHeaderTab, minSup, set([]), myFreqList)

myFreqList

   Null Set   1
     z   5
       r   1
         p   1
       x   3
         t   3
           s   2
             y   2
               q   1
           r   1
             y   1
               q   1
                 p   1
     x   1
       s   1
         r   1
finalFrequent Item:  {'p'}
condPattBases : p {frozenset({'z', 'r'}): 1, frozenset({'q', 'z', 'x', 't', 'r', 'y'}): 1}
conditional tree for:  {'p'}
   Null Set   1
     z   2
       r   2
finalFrequent Item:  {'z', 'p'}
condPattBases : z {}
finalFrequent Item:  {'p', 'r'}
condPattBases : r {frozenset({'z'}): 2}
conditional tree for:  {'p', 'r'}
   Null Set   1
     z   2
finalFrequent Item:  {'z', 'p', 'r'}
condPattBases : z {}
finalFrequent Item:  {'q'}
condPattBases : q {frozenset({'z', 'x', 't', 'r', 'y'}): 1, frozenset({'z', 'x', 't', 's', 'y'}): 1}
conditional tree for:  {'q'}
   Null Set   1
     z   2
       x   2
         t   2
           y   2
finalFrequent Item:  {'q', 'z'}
condPattBases : z {}
finalFrequent Item:  {'q', '

[{'p'},
 {'p', 'z'},
 {'p', 'r'},
 {'p', 'r', 'z'},
 {'q'},
 {'q', 'z'},
 {'q', 'x'},
 {'q', 'x', 'z'},
 {'q', 't'},
 {'q', 't', 'z'},
 {'q', 't', 'x'},
 {'q', 't', 'x', 'z'},
 {'q', 'y'},
 {'q', 'x', 'y'},
 {'q', 'y', 'z'},
 {'q', 'x', 'y', 'z'},
 {'q', 't', 'y'},
 {'q', 't', 'y', 'z'},
 {'q', 't', 'x', 'y'},
 {'q', 't', 'x', 'y', 'z'},
 {'r'},
 {'r', 'z'},
 {'r', 'x'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'s'},
 {'s', 'z'},
 {'s', 'x', 'z'},
 {'s', 't'},
 {'s', 't', 'z'},
 {'s', 't', 'x'},
 {'s', 't', 'x', 'z'},
 {'s', 'x'},
 {'y'},
 {'s', 'y'},
 {'s', 'x', 'y'},
 {'s', 'y', 'z'},
 {'s', 'x', 'y', 'z'},
 {'s', 't', 'y'},
 {'s', 't', 'y', 'z'},
 {'s', 't', 'x', 'y'},
 {'s', 't', 'x', 'y', 'z'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'t', 'y'},
 {'t', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'x', 'y'},
 {'x'},
 {'x', 'z'},
 {'z'}]